# Overview

Notebook by Sky Duryee. Code authors: Ryan Critchlow, Logan Day, Theo Hytopoulos, Vivian White, Aidan Clements, Siddhu Bhimireddy, Randall J. Eck, Sky Duryee, and possibly more.

This notebook has most of the code we used to perform RNA-seq in CSCI597k for group 4: transcription factor network analysis. (Some is included in external files.)

It takes a little bit of elbow grease to get this working on anything other than our microglia samples. **Unfortunately, some of the scripts rely on the files having specific names like "01 Agrin Coating" or "0M_output.csv".**

Process:
1. Start: FASTQ files
2. RNA STAR
3. Sambamba
4. htseq count
5. DESeq2
6. ChEA3
7. Graph R script

# Things to install

Here are some things you'll need to install. (Probably incomplete.) These were all run on Linux.

* Python packages
```
requests
pandas
```
* R packages
```
readxl
ggplot2
dplyr 
ggrepel
readr
```
* Bioconductor R packages (These should be installed automatically by the scripts themselves)
```
DESeq2
org.Hs.eg.db
```
* packages from package manager (e.g. apt)
```
sambamba
```

In [ ]:
!sudo apt install sambamba

# RNA STAR

We used RNA star to convert the .fastq files into aligned .sam files. We used the human genome gtf file provided at <https://www.ensembl.org/Homo_sapiens/Info/Index?redirect=no>. Alignment takes a long time so we did it using a cluster to speed things up. I was not involved in setting this up. HTCondor was used to manage the cluster. I heard another student mention that they were planning to leave the RNA STAR code on the cluster since it might be needed in the future.

Note: I don't have access to the cluster so I tried to restructure the data directory as I think it is on the cluster.

Input dir: /cluster/academic/projects/rna-star/STAR/data (not 100% sure)
Output dir: results

Authors: Theo Hytopoulos, Vivian White, Aidan Clements, possibly 1-2 other folks I didn't record.

**Code directory: STAR**

# Sambamba

This uses Sambamba to turn the .sam files into .bam files. This is supposed to make htseq count faster.

Input dir: results
Output dir: sorted_bam

Author: Logan Day.

In [ ]:
%%bash

for dir in results/*/; do
  echo dir $dir
  if [ -d "$dir" ]; then
    for subdir in "$dir"/*/; do
      subdir_name=$(basename "$subdir")
      if [ -f "$subdir/Aligned.out.sam" ]; then
        sam_file="$subdir/Aligned.out.sam"
        group="${dir:9:3}"
        short="${subdir_name:0:12}"
        output=$group-$short
        ./sambamba view -S --format=bam $sam_file | ./sambamba sort -n -o sorted_bam/${output}.bam /dev/stdin
      fi
    done
  fi
done


# htseq count

htseq count is used to create a count matrix for each group of samples, and save it in CSV files. The files are named "0M_output.csv" for 00: Matrigel coating, "1a_output.csv" for 01: agrin, etc. Unfortunately it will break for differently-named files.

Input dir: sorted_bam
Output dir: htseqModResults

Authors: probably Vivian White, Siddhu Bhimireddy, and Logan Day.

In [ ]:
%%bash

#!/bin/bash
prefixes=("0M" "0P" "1a" "2P" "3L" "4C")
for pre in "${prefixes[@]}"; do
  bam_files=()
  for bam in sorted_bam/"$pre"*.bam; do
    bam_files+=($bam)
  done
  echo counting ${bam_files[@]}
    htseq-count --format=bam --order=name --stranded=yes "${bam_files[@]}" Homo_sapiens.GRCh38.113.gtf > htseqModResults/"$pre"_output.csv
  echo output to htseqModResults/"$pre"_output.csv
done


# DESeq2

Performs differential gene expression analysis. Filters by p-value and log2FoldChange as well. See the comment in the R code for info on adjusting filters.

I was not able to get this file to run. I got an error: `None of the keys entered are valid keys for 'ENSEMBL'.` The author, Ryan Critchlow, insists that it works on his machine.

Input dir: htseqModResults
Output dir: filteredResults

Author: Ryan Critchlow

In [1]:
%load_ext rpy2.ipython

In [ ]:
%%R

#####
# adjust the lft_thresh (log2FoldChange threshold)
# and padj_thresh (adjusted p-value threshold)
# in the filter_results function. (You can ctrl+F for this)
##### 

.libPaths(".")

# Install BiocManager (if not already installed)
if (!requireNamespace("BiocManager", quietly = TRUE))
  install.packages("BiocManager")

# Install DESeq2 from Bioconductor
BiocManager::install("DESeq2", force = TRUE, lib = "packages")

# Load DESeq2 library
library(DESeq2)

# used for count matrix
library(dplyr)

if (!requireNamespace("org.Hs.eg.db", quietly = TRUE)) {
  BiocManager::install("org.Hs.eg.db", lib = "packages", force = TRUE)
}
library(org.Hs.eg.db)

# Function to map Ensembl gene IDs to gene symbols
add_gene_symbols <- function(data) {
  gene_ids <- rownames(data)
  annots <- select(org.Hs.eg.db,
                   keys = gene_ids,
                   columns = "SYMBOL",
                   keytype = "ENSEMBL")
  data <- merge(data,
                annots,
                by.x = "row.names",
                by.y = "ENSEMBL",
                all.x = TRUE)
  rownames(data) <- data$Row.names
  data <- data[, -1]  # Remove redundant Row.names column
  return(data)
}

run_individual_tests <- function(count_matrix,
                                 treatments,
                                 colData,
                                 control_label_0M,
                                 control_label_0P) {
  # Ensure condition is a factor
  colData$condition <- as.factor(colData$condition)

  # Initialize DESeq2 dataset
  dds <- DESeqDataSetFromMatrix(countData = count_matrix,
                                colData = colData,
                                design = ~ condition)

  # filter out rows with low gene counts
  dds <- dds[rowSums(counts(dds)) > 1, ]

  # Run DESeq2
  dds <- DESeq(dds)

  # Store results for each treatment
  results_list <- list()

  # Filter function without additional libraries
  filter_results <- function(res_df,
                             lfc_thresh = 1,
                             padj_thresh = 0.05) {
    # Remove rows with NA values in log2FoldChange or padj
    res_df <- res_df[!is.na(res_df$log2FoldChange) &
                       !is.na(res_df$padj), ]
    # filter by threshold values
    res_df <- res_df[abs(res_df$log2FoldChange) > lfc_thresh &
                       res_df$padj < padj_thresh, ]
    return(res_df)
  }

  # Run comparisons for treatments vs. 0M control
  for (treatment in treatments) {
    res <- results(dds, contrast = c("condition", treatment, control_label_0M))
    res_df <- as.data.frame(res)  # Convert to data frame
    filtered_res <- filter_results(res_df)  # Filter results
    # Add gene symbols
    filtered_res <- add_gene_symbols(filtered_res)
    results_list[[paste0(treatment, "_vs_", control_label_0M)]] <- filtered_res
  }

  # Run comparisons for treatments vs. 0P control
  for (treatment in treatments) {
    res <- results(dds, contrast = c("condition", treatment, control_label_0P))
    res_df <- as.data.frame(res)  # Convert to data frame
    filtered_res <- filter_results(res_df)  # Filter results
    # Add gene symbols
    filtered_res <- add_gene_symbols(filtered_res)
    results_list[[paste0(treatment, "_vs_", control_label_0P)]] <- filtered_res
  }

  return(results_list)
}

# Define the function
load_count_matrix <- function() {
  # File paths for the counts files
  counts_files <- list(
    "0M" = "htseqModResults/0M_output.csv",
    "0P" = "htseqModResults/0P_output.csv",
    "1A" = "htseqModResults/1a_output.csv",
    "2P" = "htseqModResults/2P_output.csv",
    "3L" = "htseqModResults/3L_output.csv",
    "4C" = "htseqModResults/4C_output.csv"
  )

  # Read all files into a list of data frames
  dataframes <- lapply(names(counts_files), function(sample) {
    file_path <- counts_files[[sample]]
    df <- read.csv(
      file_path,
      sep = "\t",
      header = FALSE,
      stringsAsFactors = FALSE
    )
    colnames(df) <- c("geneID", paste0(sample, "_col", seq(1, ncol(df) - 1)))
    return(df)
  })

  # Merge all data frames on "geneID" using Reduce and full_join
  count_matrix <- Reduce(function(x, y) {
    full_join(x, y, by = "geneID")
  }, dataframes)

  # Replace NA values with zeros
  count_matrix[is.na(count_matrix)] <- 0

  # Filter out rows (genes) with zero counts across all samples
  count_matrix <- count_matrix[rowSums(count_matrix[, -1]) > 0, ]

  # Filter gene IDs that start with 'E'
  count_matrix <- count_matrix[grepl("^E", count_matrix$geneID), ]

  # Set geneID as rownames and remove the geneID column
  rownames(count_matrix) <- count_matrix$geneID
  count_matrix <- count_matrix[, -1]  # Drop the geneID column

  return(count_matrix)
}

# obtain count matrix
count_matrix <- load_count_matrix()

count_matrix <- round(count_matrix)  # Ensure counts are integers

# Define metadata
colData <- data.frame(
  condition = c(
    rep("0M", 8),
    rep("0P", 8),
    rep("1A", 6),
    rep("2P", 8),
    rep("3L", 8),
    rep("4C", 8)
  ),
  row.names = colnames(count_matrix)
)

all(colnames(count_matrix) %in% rownames (colData))

all(colnames(count_matrix) == rownames(colData))

# Define treatments
treatments <- c("1A", "2P", "3L", "4C")

output_dir <- "filteredResults"

# run the tests
results <- run_individual_tests(
  count_matrix = count_matrix,
  treatments = treatments,
  colData = colData,
  control_label_0M = "0M",
  control_label_0P = "0P"
)

# Inspect results and print to files
for (name in names(results)) {
  cat("\nComparison:", name, "\n")
  print(results[[name]])
  # Write filtered results to CSV in the downloads directory
  write.csv(results[[name]],
            file = file.path(output_dir
                             , paste0(name, "_filtered.csv")),
            row.names = TRUE)
}


* installing *source* package ‘DESeq2’ ...
** using staged installation
** libs


g++ -std=gnu++14 -I"/usr/share/R/include" -DNDEBUG  -I'/home/ahdog/R/x86_64-pc-linux-gnu-library/4.1/Rcpp/include' -I'/home/ahdog/R/x86_64-pc-linux-gnu-library/4.1/RcppArmadillo/include'    -fpic  -g -O2 -ffile-prefix-map=/build/r-base-4A2Reg/r-base-4.1.2=. -fstack-protector-strong -Wformat -Werror=format-security -Wdate-time -D_FORTIFY_SOURCE=2 -g  -c DESeq2.cpp -o DESeq2.o
g++ -std=gnu++14 -I"/usr/share/R/include" -DNDEBUG  -I'/home/ahdog/R/x86_64-pc-linux-gnu-library/4.1/Rcpp/include' -I'/home/ahdog/R/x86_64-pc-linux-gnu-library/4.1/RcppArmadillo/include'    -fpic  -g -O2 -ffile-prefix-map=/build/r-base-4A2Reg/r-base-4.1.2=. -fstack-protector-strong -Wformat -Werror=format-security -Wdate-time -D_FORTIFY_SOURCE=2 -g  -c RcppExports.cpp -o RcppExports.o
g++ -std=gnu++14 -shared -L/usr/lib/R/lib -Wl,-Bsymbolic-functions -flto=auto -ffat-lto-objects -flto=auto -Wl,-z,relro -o DESeq2.so DESeq2.o RcppExports.o -llapack -lblas -lgfortran -lm -lquadmath -L/usr/lib/R/lib -lR


installing to /home/ahdog/R/x86_64-pc-linux-gnu-library/4.1/00LOCK-DESeq2/00new/DESeq2/libs
** R
** inst
** byte-compile and prepare package for lazy loading
** help
*** installing help indices
** building package indices
** installing vignettes
** testing if installed package can be loaded from temporary location
** checking absolute paths in shared objects and dynamic libraries
** testing if installed package can be loaded from final location
** testing if installed package keeps a record of temporary installation path
* DONE (DESeq2)


'getOption("repos")' replaces Bioconductor standard repositories, see
'help("repositories", package = "BiocManager")' for details.
Replacement repositories:
    CRAN: https://cloud.r-project.org
Bioconductor version 3.14 (BiocManager 1.30.25), R 4.1.2 (2021-11-01)
Installing package(s) 'DESeq2'
Warning in install.packages(...) : 'lib = "packages"' is not writable
trying URL 'https://bioconductor.org/packages/3.14/bioc/src/contrib/DESeq2_1.34.0.tar.gz'
Content type 'application/octet-stream' length 2088431 bytes (2.0 MB)
downloaded 2.0 MB


The downloaded source packages are in
	‘/tmp/RtmpMMOpO0/downloaded_packages’
converting counts to integer mode
estimating size factors
estimating dispersions
gene-wise dispersion estimates
mean-dispersion relationship
-- note: fitType='parametric', but the dispersion trend was not well captured by the
   function: y = a/x + b, and a local regression fit was automatically substituted.
   specify fitType='local' or 'mean' to avoid this message next tim

RInterpreterError: Failed to parse and evaluate line '\n.libPaths(".")\n\n# Install BiocManager (if not already installed)\nif (!requireNamespace("BiocManager", quietly = TRUE))\n  install.packages("BiocManager")\n\n# Install DESeq2 from Bioconductor\nBiocManager::install("DESeq2", force = TRUE, lib = "packages")\n\n# Load DESeq2 library\nlibrary(DESeq2)\n\n# used for count matrix\nlibrary(dplyr)\n\nif (!requireNamespace("org.Hs.eg.db", quietly = TRUE)) {\n  BiocManager::install("org.Hs.eg.db", lib = "packages", force = TRUE)\n}\nlibrary(org.Hs.eg.db)\n\n# Function to map Ensembl gene IDs to gene symbols\nadd_gene_symbols <- function(data) {\n  gene_ids <- rownames(data)\n  annots <- select(org.Hs.eg.db,\n                   keys = gene_ids,\n                   columns = "SYMBOL",\n                   keytype = "ENSEMBL")\n  data <- merge(data,\n                annots,\n                by.x = "row.names",\n                by.y = "ENSEMBL",\n                all.x = TRUE)\n  rownames(data)

## Filtering output files (if above script does not work)
The above R code filters the files. Since I was not able to get it to run, Ryan used his above code to generate some unfiltered results and sent them to me. (He adjusted the parameters not to filter anything out.) Then I used this script to filter out his results.

You can skip running this if you get the above R code to run.

Input dir: unfilteredResults (`src_dir` variable)
Output dir: pyFilteredResults (`dst_dir` variable)

Author: Sky Duryee

In [ ]:
import os
from pathlib import Path
import pandas as pd

src_dir = "unfilteredResults"
dst_dir = "pyFilteredResults"

padj_thresh = 0.05
log2FoldChange_thresh = 0.5

Path(dst_dir).mkdir(exist_ok=True)
for unfiltered_file in os.listdir(src_dir):
    print(f"Opened: {unfiltered_file}")
    result = pd.read_csv(f"{src_dir}/{unfiltered_file}")
    # filter using adjust p-value < 0.05 and log2FoldChange > 0.5
    result = result[(result["padj"] < padj_thresh) & (abs(result["log2FoldChange"]) > log2FoldChange_thresh)]
    result = result.sort_values("log2FoldChange")

    # print(result)
    result.to_csv(f"{dst_dir}/{unfiltered_file}", header=True)

Opened: 1A_vs_0P_filtered.csv
Opened: 2P_vs_0M_filtered.csv
Opened: 3L_vs_0M_filtered.csv
Opened: 4C_vs_0P_filtered.csv
Opened: 2P_vs_0P_filtered.csv
Opened: 1A_vs_0M_filtered.csv
Opened: 4C_vs_0M_filtered.csv
Opened: 3L_vs_0P_filtered.csv


# ChEA3
Run ChEA3 using genes from the filtered files.

This relies on ChEA3's API, so it might not work if their website goes down.

The input dir might need to be changed, if you adjust the parameters. See the *DESeq2* and *Filtering Output Files* sections.

Input dir: pyFilteredResults (`src_dir` variable) 
Output dir: chea3Results (`dst_dir` variable)

Author: Sky Duryee

In [3]:
# Run ChEA3 on each dataframe

import csv
import requests
import pandas as pd
import os

# The number of genes in the CSV required for ChEA3.
# (We don't want to make a request to ChEA3 with an empty gene set,
# for example.)
thresh = 2

src_dir = "pyFilteredResults"
dst_dir = "chea3Results"

# load the gene names from a CSV file.
def get_gene_set(filename) -> list:
    data = pd.read_csv(filename)

    # return symbol column
    return list(symbol for symbol in data["SYMBOL"] if isinstance(symbol, str))

# Creates a CSV file with ChEA3 using the given differentailly expressed
# gene set and output filename.
def make_chea3_tsv(gene_set, output_filename):
    if len(gene_set) < thresh:
        print(f"Gene set only has {len(gene_set)} genes. Not producing CSV.")
        return

    # The ChEA3 API URL.
    url = "https://maayanlab.cloud/chea3/api/enrich/"

    # Sends a JSON query to the ChEA3 API.
    # https://maayanlab.cloud/chea3/
    query_data = {
        "query_name":"myQuery",
        "gene_set":gene_set
    }
    result = requests.post(url, json=query_data)
    assert result.ok, f"Request failed with error code: {result.status_code}.\
    Check the ChEA3 website to make sure the request was correct."

    # ChEA3 produces several tables (objects) in its JSON response.
    # we get items from the table with this name:
    entry = "ENCODE--ChIP-seq"
    tfs = result.json()[entry]

    with open(output_filename, "w") as tsvfile:
        # Each item in the tfs is a dict. We make the table header using its keys.
        fieldnames = tfs[0].keys()

        writer = csv.DictWriter(tsvfile, fieldnames, delimiter="\t")
        writer.writeheader()
        for row in tfs:
            writer.writerow(row)

    print(f"Finished writing results to {output_filename}")

!mkdir -p {dst_dir}

for filename in os.listdir(src_dir):
    print(f"Generating TSV for {src_dir}/{filename}...")
    gene_set = get_gene_set(f"{src_dir}/{filename}")
    new_filename = filename.rstrip("_filtered.csv") + ".tsv"
    make_chea3_tsv(gene_set, f"{dst_dir}/{new_filename}")


Generating TSV for pyFilteredResults/1A_vs_0P_filtered.csv...
Finished writing results to chea3Results/1A_vs_0P.tsv
Generating TSV for pyFilteredResults/2P_vs_0M_filtered.csv...
Gene set only has 0 genes. Not producing CSV.
Generating TSV for pyFilteredResults/3L_vs_0M_filtered.csv...
Finished writing results to chea3Results/3L_vs_0M.tsv
Generating TSV for pyFilteredResults/4C_vs_0P_filtered.csv...
Finished writing results to chea3Results/4C_vs_0P.tsv
Generating TSV for pyFilteredResults/2P_vs_0P_filtered.csv...
Finished writing results to chea3Results/2P_vs_0P.tsv
Generating TSV for pyFilteredResults/1A_vs_0M_filtered.csv...
Finished writing results to chea3Results/1A_vs_0M.tsv
Generating TSV for pyFilteredResults/4C_vs_0M_filtered.csv...
Finished writing results to chea3Results/4C_vs_0M.tsv
Generating TSV for pyFilteredResults/3L_vs_0P_filtered.csv...
Finished writing results to chea3Results/3L_vs_0P.tsv


# Graphs
Uses modified R code from a previous project to create graphs for each control-treatment pair.

Input dir: chea3Results (`src_dir` variable)
Output dir: graphs (`dst_dir` variable)

Author: Randall J. Eck wrote transcriptionFactors.R. Slightly modified by Sky Duryee.

In [4]:
import os

src_dir = "chea3Results"
dst_dir = "graphs"

!mkdir -p graphs

for file in os.listdir(src_dir):
    print(f"Generating graph for {file}...")
    assert file.endswith(".tsv"), f"Only TSV files should be in the\
     {src_dir} folder"

    base = os.path.basename(file).rstrip(".tsv")

    !Rscript transcriptionFactors.R {src_dir}/{file} {dst_dir}/{base}.png

Generating graph for 4C_vs_0P.tsv...
chea3Results/4C_vs_0P.tsv graphs/4C_vs_0P.png[1] "Reading from chea3Results/4C_vs_0P.tsv printing to graphs/4C_vs_0P.png"
Rows: 118 Columns: 12
── Column specification ────────────────────────────────────────────────────────
Delimiter: "\t"
chr (5): Query Name, Set_name, TF, Library, Overlapping_Genes
dbl (7): Rank, Scaled Rank, Intersect, Set length, FET p-value, FDR, Odds Ratio

ℹ Use `spec()` to retrieve the full column specification for this data.
ℹ Specify the column types or set `show_col_types = FALSE` to quiet this message.
Generating graph for 3L_vs_0M.tsv...
chea3Results/3L_vs_0M.tsv graphs/3L_vs_0M.png[1] "Reading from chea3Results/3L_vs_0M.tsv printing to graphs/3L_vs_0M.png"
Rows: 118 Columns: 12
── Column specification ────────────────────────────────────────────────────────
Delimiter: "\t"
chr (5): Query Name, Set_name, TF, Library, Overlapping_Genes
dbl (7): Rank, Scaled Rank, Intersect, Set length, FET p-value, FDR, Odds Ratio

ℹ Us